In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import acquire

Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.\
Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.\
Only include properties that include a latitude and longitude value.

In [93]:
df = acquire.get_zillow_data()

In [94]:
df.head(2)

,typeconstructiontypeid,airconditioningtypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,parcelid,id,logerror,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,airconditioningdesc,typeconstructiondesc
0,nan,nan,nan,261.00,nan,nan,nan,14297519,0,0.03,...,None,nan,"60,590,630,072,012.00",None,None,None,Single Family Residential,None,None,None
1,nan,nan,nan,261.00,nan,nan,nan,17052889,1,0.06,...,None,nan,"61,110,010,023,006.00",None,None,None,Single Family Residential,None,None,None


In [95]:
df = pd.concat([
    df.iloc[:, :11], # all the rows, and up to, but not including the index of the column to drop
    df.iloc[:, 11 + 1:] # all the rows, and everything after the column to drop
], axis=1)

In [96]:
#sort values by parcelid and transactiondate
df = df.sort_values(["parcelid", "transactiondate"])

# drop duplicates
df = df.drop_duplicates(subset=["parcelid"],keep='last')

### Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 54984 to 16644
Data columns (total 68 columns):
typeconstructiontypeid          222 non-null float64
airconditioningtypeid           24953 non-null float64
storytypeid                     50 non-null float64
propertylandusetypeid           77381 non-null float64
heatingorsystemtypeid           49440 non-null float64
buildingclasstypeid             15 non-null float64
architecturalstyletypeid        206 non-null float64
parcelid                        77381 non-null int64
id                              77381 non-null int64
logerror                        77381 non-null float64
transactiondate                 77381 non-null object
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
decktypeid               

In [98]:
df.shape[0]

77381

#### Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [99]:
df.head(2)

,typeconstructiontypeid,airconditioningtypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,parcelid,id,logerror,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,airconditioningdesc,typeconstructiondesc
54984,nan,nan,nan,261.00,2.00,nan,nan,10711855,55006,-0.01,...,None,nan,"60,371,132,321,007.00",None,None,Central,Single Family Residential,None,None,None
71352,nan,1.00,nan,261.00,2.00,nan,nan,10711877,71382,0.02,...,None,nan,"60,371,132,321,007.00",None,None,Central,Single Family Residential,None,Central,None


In [106]:
def rows_missing(df):
    df1 = pd.DataFrame(df.isnull().sum(), columns = ['num_rows_missing'])
    df1['pct_rows_missing'] = df1.num_rows_missing/df.shape[0]
    return df1  

In [107]:
df1 = rows_missing(df)
df1

,num_rows_missing,pct_rows_missing
typeconstructiontypeid,77159,1.00
airconditioningtypeid,52428,0.68
storytypeid,77331,1.00
propertylandusetypeid,0,0.00
heatingorsystemtypeid,27941,0.36
...,...,...
heatingorsystemdesc,27941,0.36
propertylandusedesc,0,0.00
storydesc,77331,1.00
airconditioningdesc,52428,0.68


##### Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [108]:
def cols_missing(df):
    df['total'] = df.apply(lambda row: row.isnull().sum(), axis =1)
    df2 = df[['total']]
    df2 = df2.reset_index()
    df2 = df2.groupby('total').count()
    df2= df2.reset_index()
    df2.rename(columns = {'total': 'num_cols_missing', 'index': 'num_rows'}, inplace = True)
    df2['pct_cols_missing'] = df2.num_cols_missing/df.shape[1]
    return df2

In [109]:
df2 = cols_missing(df)

In [111]:
df2.head(2)

,num_cols_missing,num_rows,pct_cols_missing
0,23,2,0.33
1,24,13,0.35


## Prepare

Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [112]:
def single_units(df):
    df = df[(df.propertylandusedesc=='Single Family Residential') & ((df.unitcnt<=1)|df.unitcnt.isnull()) & (df.bedroomcnt>0)\
           & (df.bathroomcnt>0) & (df.calculatedfinishedsquarefeet>350)]
    return df

In [113]:
df = single_units(df)

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
A dataframe\
A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).\
A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).

In [114]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

Decide how to handle the remaining missing values:

Fill with constant value.\
Impute with mean, median, mode.\
Drop row/column

In [115]:
df = handle_missing_values(df)

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52131 entries, 54984 to 35541
Data columns (total 35 columns):
propertylandusetypeid           52131 non-null float64
heatingorsystemtypeid           33807 non-null float64
parcelid                        52131 non-null int64
id                              52131 non-null int64
logerror                        52131 non-null float64
transactiondate                 52131 non-null object
bathroomcnt                     52131 non-null float64
bedroomcnt                      52131 non-null float64
buildingqualitytypeid           33603 non-null float64
calculatedbathnbr               52116 non-null float64
calculatedfinishedsquarefeet    52131 non-null float64
finishedsquarefeet12            51973 non-null float64
fips                            52131 non-null float64
fullbathcnt                     52116 non-null float64
latitude                        52131 non-null float64
longitude                       52131 non-null float64
lotsizesquar

Decide how to handle the remaining missing values:

Fill with constant value.\
Impute with mean, median, mode.\
Drop row/column

In [117]:
%%time
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=3)

knn_imputer.fit(df.select_dtypes(exclude=[object]))



CPU times: user 27.2 ms, sys: 10.4 ms, total: 37.6 ms
Wall time: 49.6 ms


KNNImputer(add_indicator=False, copy=True, metric='nan_euclidean',
           missing_values=nan, n_neighbors=3, weights='uniform')

In [118]:
%%time
df3 = pd.DataFrame(knn_imputer.transform(df.select_dtypes(exclude=[object])), columns = df.select_dtypes(exclude='object').columns, index = df.index)

CPU times: user 2min 17s, sys: 34.8 s, total: 2min 52s
Wall time: 3min 43s


In [119]:
df3

,propertylandusetypeid,heatingorsystemtypeid,parcelid,id,logerror,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,...,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,total
54984,261.00,2.00,"10,711,855.00","55,006.00",-0.01,2.00,3.00,8.00,2.00,"2,107.00",...,0.00,1.00,"1,972.00","249,655.00","624,139.00","2,016.00","374,484.00","7,659.36","60,371,132,321,007.00",31.00
71352,261.00,2.00,"10,711,877.00","71,382.00",0.02,2.00,4.00,8.00,2.00,"1,882.00",...,0.00,1.00,"1,972.00","253,000.00","660,000.00","2,016.00","407,000.00","8,123.91","60,371,132,321,007.00",29.00
23196,261.00,2.00,"10,711,888.00","23,209.00",0.08,2.00,4.00,8.00,2.00,"1,882.00",...,0.00,1.00,"1,972.00","257,591.00","542,923.00","2,016.00","285,332.00","6,673.24","60,371,132,321,007.00",31.00
18007,261.00,2.00,"10,711,910.00","18,017.00",-0.04,2.00,3.00,8.00,2.00,"1,477.00",...,0.00,1.00,"1,960.00","57,968.00","78,031.00","2,016.00","20,063.00","1,116.46","60,371,132,321,008.00",31.00
20366,261.00,2.00,"10,711,923.00","20,378.00",-0.01,2.00,4.00,8.00,2.00,"1,918.00",...,0.00,1.00,"1,960.00","167,869.00","415,459.00","2,016.00","247,590.00","5,239.85","60,371,132,321,008.00",31.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54766,261.00,2.00,"167,636,727.00","54,788.00",0.02,3.00,4.00,4.00,3.00,"2,670.00",...,0.00,1.00,"2,006.00","182,200.00","242,900.00","2,016.00","60,700.00","3,865.75","61,110,060,061,057.00",36.00
4178,261.00,7.00,"167,637,054.00","4,183.00",-0.22,1.00,2.00,7.00,1.00,756.00,...,0.00,1.00,"1,953.00","20,083.00","84,683.00","2,016.00","64,600.00","1,631.15","61,110,060,061,057.00",38.00
62547,261.00,2.00,"167,637,371.00","62,572.00",2.07,5.00,4.00,8.00,5.00,"4,000.00",...,0.00,1.00,"2,013.00","1,664,376.00","2,597,367.00","2,016.00","932,991.00","54,391.29","61,110,046,394,719.00",35.00
46143,261.00,2.00,"167,638,003.00","46,162.00",-0.16,3.00,3.00,4.00,3.00,"2,108.00",...,0.00,1.00,"2,014.00","303,559.00","709,659.00","2,016.00","406,100.00","8,621.13","61,110,060,061,057.00",36.00


In [120]:
pd.merge(df3, df[['transactiondate', 'propertycountylandusecode','propertylandusedesc']], left_index = True, right_index = True, how = 'left')


,propertylandusetypeid,heatingorsystemtypeid,parcelid,id,logerror,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,total,transactiondate,propertycountylandusecode,propertylandusedesc
54984,261.00,2.00,"10,711,855.00","55,006.00",-0.01,2.00,3.00,8.00,2.00,"2,107.00",...,"249,655.00","624,139.00","2,016.00","374,484.00","7,659.36","60,371,132,321,007.00",31.00,2017-07-07,0101,Single Family Residential
71352,261.00,2.00,"10,711,877.00","71,382.00",0.02,2.00,4.00,8.00,2.00,"1,882.00",...,"253,000.00","660,000.00","2,016.00","407,000.00","8,123.91","60,371,132,321,007.00",29.00,2017-08-29,0101,Single Family Residential
23196,261.00,2.00,"10,711,888.00","23,209.00",0.08,2.00,4.00,8.00,2.00,"1,882.00",...,"257,591.00","542,923.00","2,016.00","285,332.00","6,673.24","60,371,132,321,007.00",31.00,2017-04-04,0100,Single Family Residential
18007,261.00,2.00,"10,711,910.00","18,017.00",-0.04,2.00,3.00,8.00,2.00,"1,477.00",...,"57,968.00","78,031.00","2,016.00","20,063.00","1,116.46","60,371,132,321,008.00",31.00,2017-03-17,0101,Single Family Residential
20366,261.00,2.00,"10,711,923.00","20,378.00",-0.01,2.00,4.00,8.00,2.00,"1,918.00",...,"167,869.00","415,459.00","2,016.00","247,590.00","5,239.85","60,371,132,321,008.00",31.00,2017-03-24,0101,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54766,261.00,2.00,"167,636,727.00","54,788.00",0.02,3.00,4.00,4.00,3.00,"2,670.00",...,"182,200.00","242,900.00","2,016.00","60,700.00","3,865.75","61,110,060,061,057.00",36.00,2017-07-07,0100,Single Family Residential
4178,261.00,7.00,"167,637,054.00","4,183.00",-0.22,1.00,2.00,7.00,1.00,756.00,...,"20,083.00","84,683.00","2,016.00","64,600.00","1,631.15","61,110,060,061,057.00",38.00,2017-01-19,0100,Single Family Residential
62547,261.00,2.00,"167,637,371.00","62,572.00",2.07,5.00,4.00,8.00,5.00,"4,000.00",...,"1,664,376.00","2,597,367.00","2,016.00","932,991.00","54,391.29","61,110,046,394,719.00",35.00,2017-07-31,0101,Single Family Residential
46143,261.00,2.00,"167,638,003.00","46,162.00",-0.16,3.00,3.00,4.00,3.00,"2,108.00",...,"303,559.00","709,659.00","2,016.00","406,100.00","8,621.13","61,110,060,061,057.00",36.00,2017-06-14,0100,Single Family Residential
